# 爬取数据

In [23]:
import pandas as pd
import urllib, os, bs4
import subprocess
import requests
import shutil
import pickle
import json
import time

def extract_wind_information(x):
    Wind_Circle = [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
    for i in range(len(x[10])):
        Wind_Circle[i] = x[10][i][1:-1]
    wind_information = [[x[4]],[x[5]],[x[6]],[x[7]],[x[8]],[x[9]],sum(Wind_Circle,[])]
    wind_information = sum(wind_information,[])
    return(wind_information)

headers = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36',
           "Referer": 'http://typhoon.nmc.cn/web.html'}

for year in range(2001,2020):
    # 爬下台风对应的id
    time_stamp = str(int(round((time.time()*1000)))) # 1970年至今毫秒数
    typhoonlist_url = 'http://typhoon.nmc.cn/weatherservice/typhoon/jsons/list_%s?t=%s&callback=typhoon_jsons_list_%s'%(year, time_stamp, year)
    res = requests.get(typhoonlist_url, headers=headers)
    typhoon_id_list = json.loads(res.text[39:-2])  # 解析json文件
    typhoon_id_dict = {x[2]:x[0] for x in typhoon_id_list} # 台风中文名
    # typhoon_id_dict2 = {x[1]:x[0] for x in typhoon_id_list} # 台风英文名
    # typhoon_id_dict.update( typhoon_id_dict2 )
    if 'NAMELESS' in typhoon_id_dict : typhoon_id_dict.pop('NAMELESS')
    if None in typhoon_id_dict : typhoon_id_dict.pop(None)
    
    for typhoon in typhoon_id_dict.keys():#['西马仑']:#
        # 爬下具体某个台风路径信息
        typhoon_id = typhoon_id_dict.get(typhoon)
        time_stamp = str(int(round((time.time()*1000))))
        typhoon_url = 'http://typhoon.nmc.cn/weatherservice/typhoon/jsons/view_%s?t=%s&callback=typhoon_jsons_view_%s'%(typhoon_id, time_stamp, typhoon_id)
        res = requests.get(typhoon_url, headers=headers)
        typhoon_tracks = json.loads(res.text[27:-1])  # 解析json文件
        台风名 = typhoon_tracks['typhoon'][1]
        台风代号 = typhoon_tracks['typhoon'][4]
        typhoon_tracks = typhoon_tracks['typhoon'][8] # 第9个数据里才有各踪迹点信息
        typhoon_tracks = {pd.to_datetime('{:0<12}'.format((x[1]))):extract_wind_information(x) for x in typhoon_tracks}
        typhoon_tracks = pd.DataFrame(typhoon_tracks).T
        typhoon_tracks.columns =['经度','纬度','中心气压(百帕)','最大风速(m/s)','移动方向','移动速度(km/h)',
                                 '七级风圈(30kts)(东北)(km)','七级风圈(30kts)(东南)(km)','七级风圈(30kts)(西南)(km)','七级风圈(30kts)(西北)(km)',
                                 '十级风圈(50kts)(东北)(km)','十级风圈(50kts)(东南)(km)','十级风圈(50kts)(西南)(km)','十级风圈(50kts)(西北)(km)',
                                 '十二级风圈(64kts)(东北)(km)','十二级风圈(64kts)(东南)(km)','十二级风圈(64kts)(西南)(km)','十二级风圈(64kts)(西北)(km)',]
        typhoon_tracks['台风名'] = 台风名
        file_path = r'E:\【论文】\【大论文】\气象资料收集\数据\中央气象台台风网数据\%s %s(中央气象台).pkl'%(台风代号,typhoon)
        with open(file_path, 'wb') as f:
            pickle.dump(typhoon_tracks, f)

# 读取数据

In [1]:
import pickle
file_path = r'E:\【论文】\【大论文】\气象资料收集\数据\中央气象台台风网数据\101 西马仑(中央气象台).pkl'
with open(file_path, 'rb') as f:
    data=pickle.load(f)
data

,经度,纬度,中心气压(百帕),最大风速(m/s),移动方向,移动速度(km/h),七级风圈(30kts)(东北)(km),七级风圈(30kts)(东南)(km),七级风圈(30kts)(西南)(km),七级风圈(30kts)(西北)(km),十级风圈(50kts)(东北)(km),十级风圈(50kts)(东南)(km),十级风圈(50kts)(西南)(km),十级风圈(50kts)(西北)(km),十二级风圈(64kts)(东北)(km),十二级风圈(64kts)(东南)(km),十二级风圈(64kts)(西南)(km),十二级风圈(64kts)(西北)(km),台风名
2001-05-09 12:00:00,120,11.8,1004,12,no,0,0,0,0,0,0,0,0,0,0,0,0,0,Cimaron
2001-05-09 18:00:00,119.6,12.3,1004,12,no,0,0,0,0,0,0,0,0,0,0,0,0,0,Cimaron
2001-05-10 00:00:00,119.5,13.1,1004,12,no,0,0,0,0,0,0,0,0,0,0,0,0,0,Cimaron
2001-05-10 06:00:00,119.3,14.1,1002,15,no,0,0,0,0,0,0,0,0,0,0,0,0,0,Cimaron
2001-05-10 12:00:00,119.1,15.3,1002,15,no,0,0,0,0,0,0,0,0,0,0,0,0,0,Cimaron
2001-05-10 18:00:00,119.1,16.2,1002,15,no,0,0,0,0,0,0,0,0,0,0,0,0,0,Cimaron
2001-05-11 00:00:00,119.1,17.1,1000,15,no,0,0,0,0,0,0,0,0,0,0,0,0,0,Cimaron
2001-05-11 06:00:00,119.1,17.6,998,18,no,0,0,0,0,0,0,0,0,0,0,0,0,0,Cimaron
2001-05-11 12:00:00,119.2,18.1,998,18,no,0,0,0,0,0,0,0,0,0,0,0,0,0,Cimaron
2001-05-11 18:00:00,119.5,18.5,995,20,no,0,0,0,0,0,0,0,0,0,0,0,0,0,Cimaron
